In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [4]:
tf.test.is_gpu_available()

True

In [5]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [6]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [8]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [9]:
#policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

In [10]:
tf.__version__

'2.0.0'

In [11]:
import pathlib
#data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
#                                         fname='flower_photos', untar=True)

val_data_dir = '/opt/AIStorage/PLAYGROUND/data/validation'
val_data_dir = pathlib.Path(val_data_dir)

data_dir = '/opt/AIStorage/PLAYGROUND/data/train'
data_dir = pathlib.Path(data_dir)

In [12]:
image_count = len(list(data_dir.glob('*/*.png')))
image_count

3196

In [13]:
val_image_count = len(list(val_data_dir.glob('*/*.png')))
val_image_count

798

In [14]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

array(['positive', 'negative'], dtype='<U8')

In [15]:
#img = list(data_dir.glob('positive/*'))

#for image_path in img[:4]:
    #display.display(Image.open(str(image_path)))

In [16]:
# The 1./255 is to convert from uint8 to float32 in range [0,1].
val_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                    #rescale=1./255,
                    #rotation_range=15,
                    #horizontal_flip=True,
                    #zoom_range=0.0
                    #)

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255
                    )

In [17]:
BATCH_SIZE = 8
#IMG_HEIGHT = 224
#IMG_WIDTH = 224
IMG_HEIGHT = 512
IMG_WIDTH = 512
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
VAL_STEPS_PER_EPOCH = np.ceil(val_image_count/BATCH_SIZE)

In [18]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

validation_data_gen = val_image_generator.flow_from_directory(directory=str(val_data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

Found 3196 images belonging to 2 classes.
Found 798 images belonging to 2 classes.


In [19]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(20,20))
  for n in range(4):
      ax = plt.subplot(2,2,n+1)
      plt.imshow(image_batch[n])
      #plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('on')

In [20]:
#image_batch, label_batch = next(train_data_gen)
#show_batch(image_batch, label_batch)

In [21]:
#label_batch

In [22]:
#val_image_batch, val_label_batch = next(validation_data_gen)
#show_batch(val_image_batch, val_label_batch)

In [23]:
#val_label_batch

In [24]:
from tensorflow.keras.callbacks import Callback

In [25]:
class LRFinder(Callback):
    """Callback that exponentially adjusts the learning rate after each training batch between start_lr and
    end_lr for a maximum number of batches: max_step. The loss and learning rate are recorded at each step allowing
    visually finding a good learning rate as per https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html via
    the plot method.
    """

    def __init__(self, start_lr: float = 1e-7, end_lr: float = 10, max_steps: int = 400, smoothing=0.9):
        super(LRFinder, self).__init__()
        self.start_lr, self.end_lr = start_lr, end_lr
        self.max_steps = max_steps
        self.smoothing = smoothing
        self.step, self.best_loss, self.avg_loss, self.lr = 0, 0, 0, 0
        self.lrs, self.losses = [], []

    def on_train_begin(self, logs=None):
        self.step, self.best_loss, self.avg_loss, self.lr = 0, 0, 0, 0
        self.lrs, self.losses = [], []

    def on_train_batch_begin(self, batch, logs=None):
        self.lr = self.exp_annealing(self.step)
        tf.keras.backend.set_value(self.model.optimizer.lr, self.lr)

    def on_train_batch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        step = self.step
        if loss:
            self.avg_loss = self.smoothing * self.avg_loss + (1 - self.smoothing) * loss
            smooth_loss = self.avg_loss / (1 - self.smoothing ** (self.step + 1))
            self.losses.append(smooth_loss)
            self.lrs.append(self.lr)

            if step == 0 or loss < self.best_loss:
                self.best_loss = loss

            if smooth_loss > 4 * self.best_loss or tf.math.is_nan(smooth_loss):
                self.model.stop_training = True

        if step == self.max_steps:
            self.model.stop_training = True

        self.step += 1

    def exp_annealing(self, step):
        return self.start_lr * (self.end_lr / self.start_lr) ** (step * 1. / self.max_steps)

    def plot(self):
        fig, ax = plt.subplots(1, 1)
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate (log scale)')
        ax.set_xscale('log')
        ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.0e'))
        ax.plot(self.lrs, self.losses)

In [26]:

class CosineAnnealer:
    
    def __init__(self, start, end, steps):
        self.start = start
        self.end = end
        self.steps = steps
        self.n = 0
        
    def step(self):
        self.n += 1
        cos = np.cos(np.pi * (self.n / self.steps)) + 1
        return self.end + (self.start - self.end) / 2. * cos


class OneCycleScheduler(Callback):
 

    def __init__(self, lr_max, steps, mom_min=0.85, mom_max=0.95, phase_1_pct=0.3, div_factor=25.):
        super(OneCycleScheduler, self).__init__()
        lr_min = lr_max / div_factor
        final_lr = lr_max / (div_factor * 1e4)
        phase_1_steps = steps * phase_1_pct
        phase_2_steps = steps - phase_1_steps
        
        self.phase_1_steps = phase_1_steps
        self.phase_2_steps = phase_2_steps
        self.phase = 0
        self.step = 0
        
        self.phases = [[CosineAnnealer(lr_min, lr_max, phase_1_steps), CosineAnnealer(mom_max, mom_min, phase_1_steps)], 
                 [CosineAnnealer(lr_max, final_lr, phase_2_steps), CosineAnnealer(mom_min, mom_max, phase_2_steps)]]
        
        self.lrs = []
        self.moms = []

    def on_train_begin(self, logs=None):
        self.phase = 0
        self.step = 0

        self.set_lr(self.lr_schedule().start)
        self.set_momentum(self.mom_schedule().start)
        
    def on_train_batch_begin(self, batch, logs=None):
        self.lrs.append(self.get_lr())
        self.moms.append(self.get_momentum())

    def on_train_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step >= self.phase_1_steps:
            self.phase = 1
            
        self.set_lr(self.lr_schedule().step())
        self.set_momentum(self.mom_schedule().step())
        
    def get_lr(self):
        try:
            return tf.keras.backend.get_value(self.model.optimizer.lr)
        except AttributeError:
            print('get lr error')
            return None
        
    def get_momentum(self):
        try:
            return tf.keras.backend.get_value(self.model.optimizer.momentum)
        except AttributeError:
            print('get momentum error')
            return None
        
    def set_lr(self, lr):
        try:
            tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        except AttributeError:
            print('set lr error')
            pass # ignore
        
    def set_momentum(self, mom):
        try:
            tf.keras.backend.set_value(self.model.optimizer.momentum, mom)
        except AttributeError:
            print('set momentum error')
            pass # ignore

    def lr_schedule(self):
        return self.phases[self.phase][0]
    
    def mom_schedule(self):
        return self.phases[self.phase][1]
    
    def plot(self):
        ax = plt.subplot(1, 2, 1)
        ax.plot(self.lrs)
        ax.set_title('Learning Rate')
        ax = plt.subplot(1, 2, 2)
        ax.plot(self.moms)
        ax.set_title('Momentum')

In [27]:
epochs = 6
lr = 3e-3
steps = np.ceil(image_count / BATCH_SIZE) * epochs
lr_schedule = OneCycleScheduler(lr, steps)


#optimizer = tf.keras.optimizers.RMSprop(lr=lr)
optimizer = tf.keras.optimizers.SGD(lr=lr)


In [28]:
steps

2400.0

In [29]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
base_model = tf.keras.applications.DenseNet169(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

In [30]:
base_model.summary()

Model: "densenet169"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [31]:



    
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
global_max_layer = tf.keras.layers.GlobalMaxPool2D()
flatten = tf.keras.layers.Flatten()

batch_norm_1 = tf.keras.layers.BatchNormalization()
drop_out_1 = tf.keras.layers.Dropout(0.25)
dense_layer_1 = tf.keras.layers.Dense(512,activation='relu')

batch_norm_2 = tf.keras.layers.BatchNormalization()
drop_out_2 = tf.keras.layers.Dropout(0.5)



prediction_layer = tf.keras.layers.Dense(2)

model = tf.keras.Sequential([
      base_model,
      global_average_layer,
      #global_max_layer,
      flatten,
      batch_norm_1,
      drop_out_1,
      dense_layer_1,
      batch_norm_2,
      drop_out_2,
      prediction_layer
])

    
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Model)          (None, 16, 16, 1664)      12642880  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1664)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1664)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1664)              6656      
_________________________________________________________________
dropout (Dropout)            (None, 1664)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               852480    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2

In [32]:
#lr_finder = LRFinder()

In [33]:
# _ = model.fit_generator(train_data_gen,
#                    epochs=1,
#                    validation_data=validation_data_gen, 
#                    steps_per_epoch=STEPS_PER_EPOCH, 
#                    validation_steps=VAL_STEPS_PER_EPOCH,
#                    callbacks=[lr_finder])

In [34]:
#lr_finder.plot() 

In [ ]:
history = model.fit_generator(train_data_gen,
                    epochs=epochs,
                    validation_data=validation_data_gen, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_steps=VAL_STEPS_PER_EPOCH,
                    callbacks=[lr_schedule])

Epoch 1/6
400/400 [==============================] - 329s 823ms/step - loss: 0.9759 - accuracy: 0.5352 - val_loss: 0.9773 - val_accuracy: 0.5570
Epoch 2/6
400/400 [==============================] - 324s 811ms/step - loss: 0.8543 - accuracy: 0.5859 - val_loss: 0.9725 - val_accuracy: 0.5846
Epoch 3/6
400/400 [==============================] - 317s 792ms/step - loss: 0.7271 - accuracy: 0.6330 - val_loss: 1.2535 - val_accuracy: 0.5457
Epoch 4/6
400/400 [==============================] - 328s 819ms/step - loss: 0.6788 - accuracy: 0.6566 - val_loss: 1.0611 - val_accuracy: 0.5564
Epoch 5/6
301/400 [=====================>........] - ETA: 1:05 - loss: 0.6847 - accuracy: 0.6458

In [ ]:
lr_schedule.plot()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [32]:
image_batch[0][0][6]

array([0.15399627, 0.15399627, 0.15399627], dtype=float32)

In [19]:
#Load using tf.data

In [20]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

val_list_ds = tf.data.Dataset.list_files(str(val_data_dir/'*/*'))

In [21]:
for f in list_ds.take(5):
  print(f.numpy())

b'/opt/AIStorage/PLAYGROUND/data/train/negative/2.25.1220173694489917139572089178995042798.png'
b'/opt/AIStorage/PLAYGROUND/data/train/negative/2.25.39943569472121983261799650628052841383.png'
b'/opt/AIStorage/PLAYGROUND/data/train/negative/2.25.292089496863278377124371205019916419751.png'
b'/opt/AIStorage/PLAYGROUND/data/train/positive/2.25.32700430559969432915927465016145140545.png'
b'/opt/AIStorage/PLAYGROUND/data/train/positive/2.25.111831954053833773869813754394525291700.png'


In [22]:
for f in val_list_ds.take(5):
  print(f.numpy())

b'/opt/AIStorage/PLAYGROUND/data/validation/positive/2.25.148043796131358029584431036706072285921.png'
b'/opt/AIStorage/PLAYGROUND/data/validation/negative/2.25.134359765825207226305787747241437173513.png'
b'/opt/AIStorage/PLAYGROUND/data/validation/negative/2.25.216719201632140760501910854022855071278.png'
b'/opt/AIStorage/PLAYGROUND/data/validation/negative/2.25.147768518981645575690513566047916354745.png'
b'/opt/AIStorage/PLAYGROUND/data/validation/negative/2.25.199902571622777137324886246475034902730.png'


In [23]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  label = -1
  if parts[-2] == 'negative':
    label = 0
  else:
    label = 1
  return label

In [24]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [25]:
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [26]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

val_labeled_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [27]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())
  

Image shape:  (224, 224, 3)
Label:  0


In [28]:
for image, label in val_labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (224, 224, 3)
Label:  0


In [29]:
def prepare_for_training(ds, shuffle=True, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  if shuffle:
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [30]:
train_ds = prepare_for_training(labeled_ds)

valid_ds = prepare_for_training(val_labeled_ds, shuffle=False)
#valid_ds = val_labeled_ds

image_batch, label_batch = next(iter(train_ds))

In [31]:
#show_batch(image_batch.numpy(), label_batch.numpy())

In [ ]:
class CosineAnnealer:
    
    def __init__(self, start, end, steps):
        self.start = start
        self.end = end
        self.steps = steps
        self.n = 0
        
    def step(self):
        self.n += 1
        cos = np.cos(np.pi * (self.n / self.steps)) + 1
        return self.end + (self.start - self.end) / 2. * cos


class OneCycleScheduler(Callback):
 

    def __init__(self, lr_max, steps, mom_min=0.85, mom_max=0.95, phase_1_pct=0.3, div_factor=25.):
        super(OneCycleScheduler, self).__init__()
        lr_min = lr_max / div_factor
        final_lr = lr_max / (div_factor * 1e4)
        phase_1_steps = steps * phase_1_pct
        phase_2_steps = steps - phase_1_steps
        
        self.phase_1_steps = phase_1_steps
        self.phase_2_steps = phase_2_steps
        self.phase = 0
        self.step = 0
        
        self.phases = [[CosineAnnealer(lr_min, lr_max, phase_1_steps), CosineAnnealer(mom_max, mom_min, phase_1_steps)], 
                 [CosineAnnealer(lr_max, final_lr, phase_2_steps), CosineAnnealer(mom_min, mom_max, phase_2_steps)]]
        
        self.lrs = []
        self.moms = []

    def on_train_begin(self, logs=None):
        self.phase = 0
        self.step = 0

        self.set_lr(self.lr_schedule().start)
        self.set_momentum(self.mom_schedule().start)
        
    def on_train_batch_begin(self, batch, logs=None):
        self.lrs.append(self.get_lr())
        self.moms.append(self.get_momentum())

    def on_train_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step >= self.phase_1_steps:
            self.phase = 1
            
        self.set_lr(self.lr_schedule().step())
        self.set_momentum(self.mom_schedule().step())
        
    def get_lr(self):
        try:
            return tf.keras.backend.get_value(self.model.optimizer.lr)
        except AttributeError:
            print('get lr error')
            return None
        
    def get_momentum(self):
        try:
            return tf.keras.backend.get_value(self.model.optimizer.momentum)
        except AttributeError:
            print('get momentum error')
            return None
        
    def set_lr(self, lr):
        try:
            tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        except AttributeError:
            print('set lr error')
            pass # ignore
        
    def set_momentum(self, mom):
        try:
            tf.keras.backend.set_value(self.model.optimizer.momentum, mom)
        except AttributeError:
            print('set momentum error')
            pass # ignore

    def lr_schedule(self):
        return self.phases[self.phase][0]
    
    def mom_schedule(self):
        return self.phases[self.phase][1]
    
    def plot(self):
        ax = plt.subplot(1, 2, 1)
        ax.plot(self.lrs)
        ax.set_title('Learning Rate')
        ax = plt.subplot(1, 2, 2)
        ax.plot(self.moms)
        ax.set_title('Momentum')

In [32]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
lr = 3e-3
steps = np.ceil(image_count / BATCH_SIZE) * epochs
lr_schedule = OneCycleScheduler(lr, steps)


mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
with mirrored_strategy.scope():

    base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

    feature_batch = base_model(image_batch)
    print(feature_batch.shape)
    base_model.trainable = False
    
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    feature_batch_average = global_average_layer(feature_batch)
    print(feature_batch_average.shape)

    
    dense_layer_1 = tf.keras.layers.Dense(2048,activation='relu')
    drop_out_1 = tf.keras.layers.Dropout(0.3)
    dense_layer_2 = tf.keras.layers.Dense(2048,activation='relu')
    drop_out_2 = tf.keras.layers.Dropout(0.3)
    prediction_layer = tf.keras.layers.Dense(1)
    #prediction_batch = prediction_layer(feature_batch_average)
    #print(prediction_batch.shape)

    model = tf.keras.Sequential([
      base_model,
      global_average_layer,
      dense_layer_1,
      drop_out_1,
      dense_layer_2,
      drop_out_2,
      prediction_layer
    ])

    
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


    model.summary()


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [33]:
history = model.fit(train_ds,
                    epochs=20,
                    validation_data=valid_ds, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_steps=VAL_STEPS_PER_EPOCH)


Train for 400.0 steps, validate for 100.0 steps
Epoch 1/20
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
400/400 [==============================] - 100s 250ms/step - loss: 0.7360 - accuracy: 0.5931 - val_loss: 0.7034 - val_accuracy: 0.5175
Epoch 2/20
400/400 [==============================] - 12s 30ms/step - loss: 0.6235 - accuracy: 0.6284 - val_loss: 0.6997 - val_accuracy: 0.5175
Epoch 3/20
400/400 [==============================] - 12s 30ms/step - loss: 0.6160 - accuracy: 0.6444 - val_loss: 0.7054 - val_accuracy: 0.5175
Epoch 4/20
400/400 [==============================] - 12s 30ms/step - loss: 0.6045 - accuracy: 0.6484 - val_loss: 0.7003 - val_accuracy: 0.5175
Epoch 5/20
400/400 [==============================] - 12s 30ms/step - lo